In [1]:
import tensorwrap as tf
from tensorwrap import nn
tf.__version__, tf.test.is_device_available('cuda')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


('0.0.0.3', False)

In [2]:
layer = nn.layers.Dense(units = 1)
layer1 = nn.layers.Dense(units = 100)
layer2 = nn.layers.Dense(units = 100)

In [3]:
array = tf.Variable([1, 2, 3, 4, 5], dtype = tf.float32)
array = tf.reshape(array, (-1, 1))

In [4]:
model = tf.nn.Sequential([
    layer1,
    layer2,
    layer1
])

model.compile(
    loss = tf.nn.losses.mse,
    optimizer = tf.nn.optimizers.SGD(learning_rate=0.1),
    metrics = tf.nn.losses.mae)

In [5]:
array

Array([[1.],
       [2.],
       [3.],
       [4.],
       [5.]], dtype=float32)

In [6]:
%timeit model.fit(array, array + 10, epochs = 100)

Epoch 1 complete - - - - - -  Metrics: 3122.241943359375
Epoch 2 complete - - - - - -  Metrics: 3950.46337890625
Epoch 3 complete - - - - - -  Metrics: 3946.90625
Epoch 4 complete - - - - - -  Metrics: 3744.697265625
Epoch 5 complete - - - - - -  Metrics: 3637.181640625
Epoch 6 complete - - - - - -  Metrics: 3953.3134765625
Epoch 7 complete - - - - - -  Metrics: 3744.152099609375
Epoch 8 complete - - - - - -  Metrics: 3103.67822265625
Epoch 9 complete - - - - - -  Metrics: 3767.4443359375
Epoch 10 complete - - - - - -  Metrics: 3730.104248046875
Epoch 11 complete - - - - - -  Metrics: 3636.81005859375
Epoch 12 complete - - - - - -  Metrics: 3730.691650390625
Epoch 13 complete - - - - - -  Metrics: 3692.408203125
Epoch 14 complete - - - - - -  Metrics: 3100.945556640625
Epoch 15 complete - - - - - -  Metrics: 3987.302734375
Epoch 16 complete - - - - - -  Metrics: 3719.123291015625
Epoch 17 complete - - - - - -  Metrics: 3968.750732421875
Epoch 18 complete - - - - - -  Metrics: 3661.5166

In [7]:
print(model.layers[1].trainable_variables[0].shape)

(100, 100)


In [8]:
array.shape, model.layers[0].kernel.shape

((5, 1), (100, 100))